In [ ]:
import pandas as pd
from fastai.vision.all import *
from pathlib import Path
import itertools
import progressbar

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
DANBOORU_PATH = Path('/home/lleonard/Documents/datasets/danbooru/0/danbooru2020/')
EXTENSIONS = ['png', 'jpg']

In [ ]:
import warnings
i = 0
def file_exists(images_id):
    with progressbar.ProgressBar(max_value=len(images_id)) as bar:
        def single_image_exists(image_id):
            global i
            last_4_digit = str(image_id % 1000).zfill(4)
            path_to_glob = str(DANBOORU_PATH / '512px' / last_4_digit / str(image_id)) + '.'
            globbed = list(itertools.chain(*[glob.glob(path_to_glob + ext) for ext in EXTENSIONS]))
            bar.update(i)
            i = i + 1
            if len(globbed) == 0:
                return False
            return True

        return images_id.apply(single_image_exists)
    
    

ratings = pd.read_csv('/home/lleonard/Documents/datasets/danbooru/0/danbooru2020/ratings_tags_0000.csv')
print(ratings)
ratings = ratings.sample(50000)
ratings = ratings[file_exists(ratings['id'])]
print(ratings.columns)

In [ ]:
#ratings = pd.read_csv('clean_0000.csv')

In [ ]:
ratings['tags'] = ratings[' "tags"'].str.split()
del ratings[' "tags"']
ratings['tags']

In [ ]:
ratings

In [ ]:
counter = defaultdict(int)
for tags in progressbar.progressbar(ratings['tags']):
    for tag in tags:
        counter[tag] = counter[tag] + 1
tags = pd.DataFrame(counter.items(), columns=['tag', 'count'])
# I do not want 'bad_id' and cie
tags = tags[~tags['tag'].str.startswith('bad')]
tags = tags[tags['tag'] != 'absurdres']

common_tags = tags.sort_values(by='count', ascending=False).head(50)['tag']
common_tags

In [ ]:
ratings['top_tags'] = ratings['tags'].transform(lambda row: set(row) & set(common_tags))
ratings

In [ ]:
# ratings.to_csv('clean_0000.csv', index=False)

In [ ]:
def get_x(row):
    image_id = str(row['id'])
    mod_1000 = str(row['id'] % 1000)
    path_to_glob = str(DANBOORU_PATH / '512px' / mod_1000.zfill(4) / str(image_id)) + '.'
    return list(itertools.chain(*[glob.glob(path_to_glob + ext) for ext in EXTENSIONS]))[0]
    

def get_y(row):
    return row['top_tags']

dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   item_tfms=Resize(224),
                   batch_tfms=[*aug_transforms(),Normalize()],
                   get_x = get_x, get_y = get_y)
dblock.summary(ratings)


In [ ]:
dataloaders = dblock.dataloaders(ratings, bs=128)
dataloaders.show_batch()

In [ ]:
learner = cnn_learner(dataloaders, models.vgg16_bn, 
                      metrics = [
                          partial(accuracy_multi, thresh=0.9),
                          APScoreMulti(),
                                ],
                      pretrained=True,
                      cbs=[SaveModelCallback(),MixUp()]
                     ).to_fp16()
learner.model

In [ ]:
learner.lr_find()

In [ ]:
learner.fit_one_cycle(20, 3e-2)

In [ ]:
learner.unfreeze()

In [ ]:
learner.fit_one_cycle(100, slice(3e-4, 3e-2))

In [ ]:
learner.save('danbooru_vgg_tag_classifier')

In [ ]:
learner.show_results(ds_idx=1, max_n=1, figsize=(15,11))

In [ ]:
torch.cuda.empty_cache()
dataloaders = dblock.dataloaders(ratings, bs=64)
model = model.eval().to('cuda')
data = iter(dataloaders.valid)

In [ ]:
import torchvision

item = next(data)
grid = torchvision.utils.make_grid(item[0])
show_image(grid)
preds = model(item[0])
print(preds[preds[:, 0].argmax(), :])
show_image(item[0][preds[:, 0].argmax()])
del preds
del item

In [ ]:
torch.save(learner.model, './models/danbooru_vgg_tags_classifier.pth')